In [1]:
# EXECUTING RAW SQL QUERIES
# Unfortunately, we can't do everything with pure Python and Pandas.
# Sometimes, writing and executing SQL queries directly on our databases proves advantageous. 
# After all, it's called a Structured Query Language, so it's designed to directly interact with databases.
# The term RAW SQL refers to code that we write by using SQL syntax and that's specific to a single database.

In [2]:
# WHAT'S CRUD?
# CRUD is an acronym that stands for CREATE, READ, UPDATE, DELETE.
# If you learn these four operations, you can build just about any application that uses SQL and databases.

In [3]:
import pandas as pd
import sqlalchemy

In [4]:
# Create a temporary sqlite database:
database_connection_string = 'sqlite:///'

# Create the database engine:
engine = sqlalchemy.create_engine(
    database_connection_string,
)
engine

Engine(sqlite:///)

In [5]:
# Create a Pandas DataFrame table to read into the SQL database:
stocks = pd.DataFrame({'AAPL': [1,2], 'GOOG': [3,4]})

# Use the `stocks` DataFrame for the data in the SQL database:
stocks.to_sql('stocks', engine, index=False, if_exists='replace')
display(stocks)

# Read the `stocks` DataFrame to a SQL table:
sql_stocks_df = pd.read_sql_table('stocks', con=engine)
display(sql_stocks_df)

,AAPL,GOOG
0,1,3
1,2,4


,AAPL,GOOG
0,1,3
1,2,4


In [6]:
# CRUD - CREATE DATA 
# Let's review how Pandas created our `stocks_dataframe` table.
# If we change our engine to ECHO, or display, the SQL code back in the notebook, we'll get the raw SQL that created the table.
# To do this, we'll create a new engine and set `echo=True`:
# Create the database engine
engine = sqlalchemy.create_engine(
    database_connection_string,
    echo=True
)

# Now we'll use the engine to create the `stocks_database` table again:
stocks.to_sql('stocks', engine, index=False, if_exists='replace')

2024-01-10 22:10:22,090 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("stocks")
2024-01-10 22:10:22,090 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,090 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("stocks")
2024-01-10 22:10:22,105 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-10 22:10:22,105 INFO sqlalchemy.engine.Engine 
CREATE TABLE stocks (
	"AAPL" BIGINT, 
	"GOOG" BIGINT
)


2024-01-10 22:10:22,105 INFO sqlalchemy.engine.Engine [no key 0.00119s] ()
2024-01-10 22:10:22,105 INFO sqlalchemy.engine.Engine COMMIT
2024-01-10 22:10:22,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-10 22:10:22,105 INFO sqlalchemy.engine.Engine INSERT INTO stocks ("AAPL", "GOOG") VALUES (?, ?)
2024-01-10 22:10:22,105 INFO sqlalchemy.engine.Engine [generated in 0.00123s] ((1, 3), (2, 4))
2024-01-10 22:10:22,121 INFO sqlalchemy.engine.Engine COMMIT


2

In [7]:
# As you can see, there is a lot displayed. However, we need only pay attention to a few things.
# First, the following SQL statement creates the table:
    # CREATE TABLE stocks (
        # 'AAPL' BIGINT,
        # 'GOOG' BIGINT
# )
# This the raw SQL statement - specifically, the `CREATE TABLE` statement - that creates the empty table.
# Let's closely examine the structure of this statement. 
# The code block starts with the `CREATE TABLE` SQL syntax, and the table name and a set of parentheses follows.
# Inside the parentheses is the name for each column in the table along with the data type of the information that the column will contain.
# Now consider the following basic syntax of the `CREATE TABLE` statement:
    # CREATE TABLE table_name (
        # column_name datatype,
        # column_name datatype
# )
# We can apply this syntax  to the SQL statement that created our `stocks` table.
# Our statement told the database to created a table named `stocks` that has columns named 'AAPL' and 'GOOG'.
# An unfamiliar term, `BIGINT`, represents the data type for the column values - which means big integers.
# Simply put, our code created an empty database table.
# It doesn't yet contain any data.
# The second thing that we should pay attention to in the earlier output is the code for inserting data into the table:
    # INSERT INTO stocks ('AAPL', 'GOOG') VALUES (?,?)
# And the following:
    # ((1,3), (3,4))
# These lines of code insert the data directly into the columns of our `stocks` table.
# Now that we understand the breakdown of the output, let's use raw SQL to create a new table in our database.

In [8]:
# CREATE A TABLE VIA A RAW SQL STATEMENT
# Let's first create the SQL statement as a multiline string so that we can use it more easily in Python.
# To create a multiline string in Python, we use the f-string format with triple quotes:
create_table = """
CREATE TABLE stocks(
    "AAPL" BIGINT,
    "GOOG" BIGINT
)
"""
create_table

'\nCREATE TABLE stocks(\n    "AAPL" BIGINT,\n    "GOOG" BIGINT\n)\n'

In [9]:
# Let's now change this SQL statement a bit and create a table named `delicious_stocks`.
# We'll also change the stock tickers to those for Brinker International ("EAT") and the Cheesecake Factory ("CAKE"):
create_table = """
CREATE TABLE delicious_stocks (
    "EAT" BIGINT,
    "CAKE" BIGINT
)
"""
create_table

'\nCREATE TABLE delicious_stocks (\n    "EAT" BIGINT,\n    "CAKE" BIGINT\n)\n'

In [10]:
# Now that we have the SQL strings, we can execute the SQL statement with our engine by using the `execute` function:
engine.execute(create_table)

2024-01-10 22:10:22,184 INFO sqlalchemy.engine.Engine 
CREATE TABLE delicious_stocks (
    "EAT" BIGINT,
    "CAKE" BIGINT
)

2024-01-10 22:10:22,184 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,184 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
# The preceding output indicates that the engine acknowledged the SQL statement to create the `delicious_stocks` table.
# The word `COMMIT`, which appears at the end of the second to last line, confirms that the `CREATE TABLE` command was executed.
# Next, let's verify that the `delicious_stocks` table was created:
engine.table_names()

2024-01-10 22:10:22,215 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2024-01-10 22:10:22,215 INFO sqlalchemy.engine.Engine [raw sql] ()


C:\Users\legar\AppData\Local\Temp\ipykernel_13336\3763842806.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['delicious_stocks', 'stocks']

In [12]:
# If we now read the `delicious_stocks` table into Pandas, we should get an empty DataFrame.
    # (Remember that the `delicious_stocks DataFrame has two columns, 'EAT'/'CAKE', but no data.
delish_sql_df = pd.read_sql_table('delicious_stocks', con=engine)
delish_sql_df

2024-01-10 22:10:22,247 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("delicious_stocks")
2024-01-10 22:10:22,247 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,247 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("delicious_stocks")
2024-01-10 22:10:22,262 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,262 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2024-01-10 22:10:22,262 INFO sqlalchemy.engine.Engine [raw sql] ('delicious_stocks',)
2024-01-10 22:10:22,262 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("delicious_stocks")
2024-01-10 22:10:22,262 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,262 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("delicious_stocks")
2024-01-10 22:10:22,262 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,262 INFO sqlalchemy.engine.Engine SELECT sql 

,EAT,CAKE


In [13]:
# INSERT DATA INTO A TABLE VIA A RAW SQL STATEMENT
# The last line of the preceding output confirms the empty DataFrame.
# Since this DataFrame is empty, we will now insert data next.
# To insert data into our table, we'll create a multiline string with our raw SQL statement:
insert_data = """
INSERT INTO delicious_stocks ("EAT", "CAKE") VALUES (?,?)
"""

# Let's break down this code as follows:
    # 1. The `INSERT INTO` element is the instruction for the database.
    # 2. The `delicous_stocks` element is the table to access.
    # 3. The "EAT" & "CAKE" elements are the columns to manipulate.
    # 4. The `VALUES` element is the SQL syntax that cues the database to check for new information.
    # 5. the question marks (?,?) indicaate that new information is forthcoming.
# Next, we insert the data by executing the `insert_data` query stirng with our engine.
# We then pass the values to insert as a second parameter:
engine.execute(insert_data, ((10, 30), (20, 40)))

2024-01-10 22:10:22,309 INFO sqlalchemy.engine.Engine 
INSERT INTO delicious_stocks ("EAT", "CAKE") VALUES (?,?)

2024-01-10 22:10:22,309 INFO sqlalchemy.engine.Engine [raw sql] ((10, 30), (20, 40))
2024-01-10 22:10:22,309 INFO sqlalchemy.engine.Engine COMMIT


In [14]:
# IMPORTANT:
# Instead of using the preceding parameterized code, we can simply add the values directly:
    # insert_data = """
    # INSERT INTO delicious_stocks ("EAT", "CAKE") VALUES (10,30), (20,40)
    # """
    # engine.execute(insert_data)

In [15]:
# To verify that the data was successfully inserted, we run the following code:
pd.read_sql_table('delicious_stocks', con=engine)

2024-01-10 22:10:22,357 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("delicious_stocks")
2024-01-10 22:10:22,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,357 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("delicious_stocks")
2024-01-10 22:10:22,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,357 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2024-01-10 22:10:22,357 INFO sqlalchemy.engine.Engine [raw sql] ('delicious_stocks',)
2024-01-10 22:10:22,372 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("delicious_stocks")
2024-01-10 22:10:22,372 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,372 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("delicious_stocks")
2024-01-10 22:10:22,372 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,372 INFO sqlalchemy.engine.Engine SELECT sql 

,EAT,CAKE
0,10,30
1,20,40


In [16]:
# READ DATA
# To read table data directly from the database by using raw SQL, we use a `SELECT` statement:
    # SELECT column_name, column_name, ...
    # FROM table_name
# Let's examine the structure of the SELECT statement. 
# In the preceding syntax, we can observe the instructions for the database, which consist of `SELECT` and `FROM`.
# We can also observe the needed information - namely, the column names and the table name.
# To read all the columns of data from our database, we'll use the following code:
read_all_data = """
SELECT "EAT", "CAKE" FROM delicious_stocks
"""
engine.execute(read_all_data)

2024-01-10 22:10:22,419 INFO sqlalchemy.engine.Engine 
SELECT "EAT", "CAKE" FROM delicious_stocks

2024-01-10 22:10:22,419 INFO sqlalchemy.engine.Engine [raw sql] ()


In [17]:
# This output shoes that the engine passed the `SELECT` statement request to the database.
# We can guess that the data was read, but what happens next?
# Making the request to read the data is only the first part. 
# We also need to tell the program what to do with the data once it's been read.
# More specifically, we need to make the data usable outside the SQLAlchemy database.
# To do that, we set the engine's execution of the statement equal to a variable named `results`.
# Then, we loop through each row of data that was returned as a result of the `SELECT` statement:
results = engine.execute(read_all_data)
for row in results:
    print(row)

2024-01-10 22:10:22,451 INFO sqlalchemy.engine.Engine 
SELECT "EAT", "CAKE" FROM delicious_stocks

2024-01-10 22:10:22,451 INFO sqlalchemy.engine.Engine [raw sql] ()
(10, 30)
(20, 40)


In [18]:
# Notice that our output has improved. 
# It shows the stock values within each row of the table.
# We can convert the results to another type of Python data container, such as a list:
results = engine.execute(read_all_data)
list(results)

2024-01-10 22:10:22,466 INFO sqlalchemy.engine.Engine 
SELECT "EAT", "CAKE" FROM delicious_stocks

2024-01-10 22:10:22,466 INFO sqlalchemy.engine.Engine [raw sql] ()


[(10, 30), (20, 40)]

In [19]:
# Once a list (or other type of data container) contains the results, we can use them to populate values in a DataFrame.
# From there, the options for analysis are endless.
# There is also a shortcut for selecting all the columns of data.
# Specifically, we use the asterisk (*) as a wildcard with the `SELECT` statement.
# This tells SQL to read all the columns of data in the table.
# As you will see, with slightly less code, we ge the same output.
read_all_data = """
SELECT * FROM delicious_stocks
"""
results = engine.execute(read_all_data)
list(results)

2024-01-10 22:10:22,482 INFO sqlalchemy.engine.Engine 
SELECT * FROM delicious_stocks

2024-01-10 22:10:22,482 INFO sqlalchemy.engine.Engine [raw sql] ()


[(10, 30), (20, 40)]

In [20]:
# UPDATE DATA
# Just like we sometimes update values in DataFrames by using `loc` and `iloc`, we might need to update data in our database tables.
# In SQL, we update values by using the `UPDATE` statement:
    # UPDATE table_name
    # SET column_name coditional_operator new_table_value
    # WHERE column_name conditional_operator current_table_value
# Notice that the `UPDATE` statement has a few keywords:
    # 1. The `UPDATE` keyword: This specifies the table that we want to update.
    # 2. The `SET` keyword: This sets new values in the table.
    # 3. The `WHERE` keyword: This finds existing values in the table that meet the specified condiiton.
# Let's put all this together to update our `delicious_stocks` table.
# We'll find out where any element in the "CAKE" column equals 40 and update that value to 100:
update_cake = """
UPDATE delicious_stocks
SET CAKE = 100
WHERE CAKE = 40
"""
engine.execute(update_cake)

2024-01-10 22:10:22,513 INFO sqlalchemy.engine.Engine 
UPDATE delicious_stocks
SET CAKE = 100
WHERE CAKE = 40

2024-01-10 22:10:22,513 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,513 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
# Now let's check the table to make sure the changes were made:
read_all_data = """
SELECT * from delicious_stocks
"""
results = engine.execute(read_all_data)
list(results)

2024-01-10 22:10:22,545 INFO sqlalchemy.engine.Engine 
SELECT * from delicious_stocks

2024-01-10 22:10:22,545 INFO sqlalchemy.engine.Engine [raw sql] ()


[(10, 30), (20, 100)]

In [22]:
# Notice that at the end of the output, the 40 has been replaced with 100, just as instructed.

In [23]:
# DELETE DATA
# We'll start this discusson by emphasizing the following: 
# Don't be the person who accidentally delets all the data in the company database!
# Luckily, the SQL `DELETE` statement inlcudes a `WHERE` clause to pinpoint the exact data that you want to delete.

# IMPORTANT
# To delet data from a table, you need only the table name and the `WHERE` clause.

# The `DELETE` syntax is as follows:
    # DELETE FROM table_name
    # WHERE table_name conditional_operator current_table_value
# To return to our `delicious_stocks` example, we'll delete the data where any element in the "EAT" column equals 20 by running the following code:
delete_eat = """
DELETE FROM delicious_stocks
WHERE EAT = 20
"""
engine.execute(delete_eat)

2024-01-10 22:10:22,576 INFO sqlalchemy.engine.Engine 
DELETE FROM delicious_stocks
WHERE EAT = 20

2024-01-10 22:10:22,576 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-10 22:10:22,576 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
# Notice that the output displays the SQL `DELETE` statement and then, two lines later, the confirmation via the `COMMIT` command.
# To confirm this action, let's read the table data and then display the output:
read_all_data = """
SELECT * FROM delicious_stocks
"""
results = engine.execute(read_all_data)
list(results)

2024-01-10 22:12:24,354 INFO sqlalchemy.engine.Engine 
SELECT * FROM delicious_stocks

2024-01-10 22:12:24,354 INFO sqlalchemy.engine.Engine [raw sql] ()


[(10, 30)]

In [ ]:
# Again, notice the SQL query, which is now a `SELECT` statement.
# But, the output then shows only one row of data - specifically, [(10,30)].
# We accidentally deleted the entire row that matched the condition!
# The output reflects only the values from the first row of data - specifically, [(10,30)].
# The values form the second row - [(20,100)] - have disappeared.
# LESSON LEARNED: The `DELETE` statement removes the entire row of data that matches the specified condition.
# It doesn't remove only the singel element that you intended for deletion.
# Unless your intention is to delete the entire row of data, consider using the `UPDATE` statement to change the value of the single element.
# For example, you can update the element with a value like 0 or 'N/A'.